In [1]:
########################################################################
#    July 22, 2019                                                     #
#    16:50                                                             #
#    Created by: Kunal Gehlot                                          #
########################################################################

In [2]:
import librosa
import librosa.display
import numpy as np
import os
import sys
import warnings
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.layers import Flatten, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from tqdm import tqdm
# import scipy.io.wavfile as wav
# from speechpy.feature import mfcc
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
classLabels = ('Angry', 'Fear', 'Disgust', 'Happy', 'Sad', 'Surprised', 'Neutral')

In [14]:
numLabels = len(classLabels)

In [10]:
in_shape = (39,216)

In [11]:
model = Sequential()

In [12]:
model.add(Conv2D(8, (13, 13), input_shape=(in_shape[0], in_shape[1], 1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(8, (13, 13)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Conv2D(8, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(8, (1, 1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Flatten())
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
model.add(Dense(numLabels, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam',
                           metrics=['accuracy'])
print(model.summary(), file=sys.stderr)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 27, 204, 8)        1360      
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 204, 8)        32        
_________________________________________________________________
activation_1 (Activation)    (None, 27, 204, 8)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 192, 8)        10824     
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 192, 8)        32        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 192, 8)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 192, 8)         0         
__________

None


In [16]:
model.load_weights('VoiceEmo.h5')

In [17]:
file = 'output.wav'

In [18]:
X, sample_rate = librosa.load(file, res_type='kaiser_best',duration=2.5,sr=22050*2,offset=0.5)
sample_rate = np.array(sample_rate)
mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=39)
feature = mfccs

In [19]:
feature = feature.reshape(39, 216, 1)

In [20]:
classLabels[np.argmax(model.predict(np.array([feature])))]

'Fear'